## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
from glob import glob
from scipy.spatial.distance import jensenshannon

## Initialize Functions

In [ ]:
# Function to compute Node Distance Distribution
def ndd(x):
    result = x.copy()
    total = np.sum(x[2])
    ndd = x[2]/total
    result[2] = ndd
    return result

# Function to generate Transition Matrix
def transition_matrix(x, metaboliteList, walk=1):
    g = x.copy()
    result = pd.DataFrame({0: [], 1: [], 2: []})
    if walk == 1:
        for i in metaboliteList:
            row_i = g[g[0] == i]
            row_sum = np.sum(row_i[2])
            if row_sum == 0:
                row_sum = 1
            for j in row_i[1]:
                val = row_i[row_i[1] == j].iat[0,2]/row_sum
                out = pd.DataFrame({0: [i], 1: [j], 2: [val]})
                result = pd.concat([result, out])
    elif walk == 2:
        walk_distances = pd.DataFrame({0: [], 1: [], 2: []})
        for i in metaboliteList:
            row_i = g[g[0] == i]
            for j in row_i[1]:
                row_j = g[g[0] == j]
                for k in row_j[1]:
                    val1 = row_i[row_i[1] == j].iat[0, 2]
                    val2 = row_j[row_j[1] == k].iat[0, 2]
                    val = [val1, val2]
                    wd = pd.DataFrame({0: [i], 1: [k], 2: [np.sum(val)]})
                    walk_distances = pd.concat([walk_distances, wd])
        for i in metaboliteList:
            row_i = walk_distances[walk_distances[0] == i]
            row_sum = np.sum(row_i[2])
            if row_sum == 0:
                row_sum = 1
            for j in row_i[1]:
                val = row_i[row_i[1] == j].iat[0,2]/row_sum
                out = pd.DataFrame({0: [i], 1: [j], 2: [val]})
                result = pd.concat([result, out])
    else:
        print('This function limited to walk = 2')
    return result

# Function to compute Kullback-Leibler Divergence
def D(p, q):
    result = 0
    for i in range(len(p)):
        if q[i] != 0:
            result += p[i]*np.log(p[i]/q[i])
    return result

# Function to compute Jensen-Shannon Divergence
def J(p, q):
    m = (p+q)/2
    DPM = D(p, m)
    DQM = D(q, m)
    return np.abs(DPM/2 + DQM/2)

# Function for Distance Measure
def M(Gp, Gq, metaboliteList):
    result = []
    for i in metaboliteList:
        Gp_i = Gp[Gp[0]==i][2].to_numpy()
        Gq_i = Gq[Gq[0]==i][2].to_numpy()
        result.append(np.sqrt(J(Gp_i, Gq_i))/(2*np.sqrt(np.log(2))))
    return np.mean(result)

## Import Data

In [ ]:
data = []
for i in glob('../diabetic/*.ncol'):
    data.append(pd.read_table(i, sep=' ', header=None))

In [ ]:
data[0]

,0,1,2
0,m01570s,m02956s,1.521454e+00
1,m01570s,m01569s,1.521454e+00
2,m02956s,m00234s,3.992614e+00
3,m02956s,m01807s,3.992614e+00
4,m00234s,m01807s,4.867763e-01
...,...,...,...
9205,m01383s,m01383c,4.074718e+00
9206,m01383s,m02519c,4.074718e+00
9207,m01383s,m01442c,4.074718e+00
9208,m00536c,m00536s,1.000000e-08


## Generate Metabolite List

In [ ]:
metaboliteList = []
metaboliteListAll = []

for i in data[0][0]:
    metaboliteList.append(i)
    metaboliteListAll.append(i)
for i in data[0][1]:
    metaboliteListAll.append(i)

metaboliteList = np.unique(metaboliteList)
metaboliteListAll = np.unique(metaboliteListAll)

In [ ]:
len(metaboliteList), len(metaboliteListAll)

(3089, 4034)

## Compute Node Distance Distribution (NDD)

In [ ]:
ndds = []
for i in data:
    ndds.append(ndd(i))

## Compute Transition Matrix (Walk 1)

In [ ]:
tm_walk_1 = []
for i in data:
    tm_walk_1.append(transition_matrix(i, metaboliteList))

## Compute Transition Matrix (Walk 2)

In [ ]:
tm_walk_2 = []
for i in data:
    tm_walk_2.append(transition_matrix(i, metaboliteList, 2))

## Compute M1

In [ ]:
ndd_distances = []
for i in range(len(ndds)):
    for j in range(i+1, len(ndds)):
        Gp = ndds[i]
        Gq = ndds[j]
        ndd_distances.append(M(Gp, Gq, metaboliteList))

In [ ]:
ndd_distances

[0.0015428613635351735,
 0.0010820617371634549,
 0.0019439215689642647,
 0.0013580351786734177,
 0.0016698592735454685,
 0.0017598024399601007]

## Compute M2

In [ ]:
tm_walk_1_distances = []
for i in range(len(tm_walk_1)):
    for j in range(i+1, len(tm_walk_1)):
        Gp = tm_walk_1[i]
        Gq = tm_walk_1[j]
        tm_walk_1_distances.append(M(Gp, Gq, metaboliteList))

In [ ]:
tm_walk_1_distances

[0.019084974856780346,
 0.018990941312785344,
 0.022908882826348956,
 0.019083756414244462,
 0.022113739438721424,
 0.02559344502218521]

## Compute M3

In [ ]:
tm_walk_2_distances = []
for i in range(len(tm_walk_2)):
    for j in range(i+1, len(tm_walk_2)):
        Gp = tm_walk_2[i]
        Gq = tm_walk_2[j]
        tm_walk_2_distances.append(M(Gp, Gq, metaboliteList))

In [ ]:
tm_walk_2_distances

[0.05044758988198939,
 0.0462254638581373,
 0.06708558715496694,
 0.049037576637369296,
 0.05851540372014448,
 0.06400187876787704]

## Graph Distance

In [ ]:
M1 = np.array(ndd_distances)
M2 = np.array(tm_walk_1_distances)
M3 = np.array(tm_walk_2_distances)
D2 = M1 + M2
D3 = M1 + M2 + M3

In [ ]:
D2

array([0.02062784, 0.020073  , 0.0248528 , 0.02044179, 0.0237836 ,
       0.02735325])

In [ ]:
D3

array([0.07107543, 0.06629847, 0.09193839, 0.06947937, 0.082299  ,
       0.09135513])

## Trial

In [ ]:
out = pd.DataFrame({0: [], 1: [], 2: []})
for i in metaboliteList:
    x = data[0][data[0][0] == i]
    for j in x[1]:
        val1 = [x[x[1] == j].iat[0, 2]]
        y = data[0][data[0][0] == j]
        for k in y[1]:
            val2 = val1.copy()
            val2.append(y[y[1] == k].iat[0, 2])
            df = pd.DataFrame({0: [i], 1: [k], 2: [np.sum(val2)]})
            out = pd.concat([out, df])
out

,0,1,2
0,m00003c,m00003c,2.481540e+00
0,m00003c,m00003c,2.345738e+00
0,m00003c,m02102c,4.434610e-01
0,m00003c,m00438c,2.438397e-01
0,m00003c,m03114c,2.000000e-08
...,...,...,...
0,m03157s,m03157s,8.512286e+00
0,m03158c,m01189c,7.706674e+00
0,m03158c,m02343c,3.401242e+01
0,m03158c,m02343c,3.401242e+01
